In [ ]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np
import datetime

# Vizualization Libraries
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# pre-processing
from sklearn.preprocessing import StandardScaler

# ML model Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import mean_squared_error

#sklearn
from sklearn.preprocessing import MinMaxScaler

#statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.statespace.sarimax import SARIMAX
!pip install pmdarima
from pmdarima.arima import auto_arima

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM, RepeatVector, TimeDistributed, Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



     |████████████████████████████████| 1.5MB 7.4MB/s 
     |████████████████████████████████| 8.7MB 11.7MB/s 
     |████████████████████████████████| 808kB 37.8MB/s 
     |████████████████████████████████| 2.1MB 45.2MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [ ]:
retail = pd.read_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/Train.csv')
retail_test = pd.read_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/Test.csv')
retail_data = retail.copy()
retail_test_data = retail_test.copy()

#Drop Duplicate rows
retail_data.drop_duplicates(subset=None, keep='first', inplace=True)
#Only dropped one outlier
retail_data.drop(retail_data.loc[retail_data['UnitPrice']>35000,:].index,inplace=True)

# #No missing values
# import missingno as msno
# msno.matrix(retail_test)
# print(retail_test.isna().sum())
#Seperate Categorical and Numerical Columns
cat_cols = retail_data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = retail_data.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

retail_data.head()

['InvoiceDate']
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [ ]:
retail_test_data.shape

(122049, 7)

In [ ]:
def drop_irrelavant_columns(df):
  df.drop(columns=['InvoiceNo','Description','Quantity','CustomerID','Country'],inplace=True)

def sample_by_hour_set_index(df):
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
  df.set_index('InvoiceDate',inplace=True)
  df.index = pd.to_datetime(df.index.strftime('%Y-%m-%d %H:%M:%S'))
  df.sort_index(inplace=True)
  df.resample(rule='H').mean().fillna(method ='ffill', inplace = True)

def convert_InvoiceDate_to_features(df):
  df['year'] = df.index.year
  df['month'] = df.index.month
  df['day'] = df.index.day
  df['hour'] = df.index.hour
  df['minute'] = df.index.minute
  return df


def convert_column_to_dummies(df,colname):
  dummies = pd.get_dummies(df[colname])

  for col in dummies.columns:
    df[col] = dummies[col]

  df.drop(columns=[colname],inplace=True)

def convert_unit_price_to_supervised(df):
  reframed = series_to_supervised(df['UnitPrice'].values, n_in=7, n_out=1, dropnan=True)
  df[reframed.columns] = reframed.values
  return df

def series_to_supervised(data, n_in=7, n_out=1, dropnan=True):
	n_vars = 1
	dff = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(dff.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	# for i in range(0, n_out):
	# 	cols.append(dff.shift(-i))
	# 	if i == 0:
	# 		names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
	# 	else:
	# 		names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.fillna(method='bfill', inplace=True)
	return agg

In [ ]:
retail_data.shape

(282158, 8)

In [ ]:
drop_irrelavant_columns(retail_data)
drop_irrelavant_columns(retail_test_data)

sample_by_hour_set_index(retail_data)
sample_by_hour_set_index(retail_test_data)

retail_data = convert_InvoiceDate_to_features(retail_data)
retail_test_data = convert_InvoiceDate_to_features(retail_test_data)

convert_column_to_dummies(retail_data,'StockCode')
convert_column_to_dummies(retail_test_data,'StockCode')

model_ready_data = convert_unit_price_to_supervised(retail_data)

model_ready_data.head()
model_ready_data_small = model_ready_data[:150000]

In [ ]:
# split into train and test sets
values = model_ready_data_small.values

n_train_time = int(len(model_ready_data_small)*0.75)
train = values[:n_train_time, :]
test = values[n_train_time:, :]
##test = values[n_train_time:n_test_time, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

test_X = train_X
test_y = train_y

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 

(112500, 1, 3631) (112500,) (112500, 1, 3631) (112500,)


In [ ]:
model = Sequential()
model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
#    model.add(LSTM(70))
#    model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

#CNN + LSTM
# model_cnn_lstm = Sequential()
# model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None,train_X.shape[1], train_X.shape[2])))
# model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=1)))
# model_cnn_lstm.add(TimeDistributed(Flatten()))
# model_cnn_lstm.add(LSTM(50, activation='relu'))
# model_cnn_lstm.add(Dense(1))
# model_cnn_lstm.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=70, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# # make a prediction
# yhat = model.predict(test_X)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[]))
# # invert scaling for forecast
# inv_yhat = np.concatenate((yhat, test_X[:, -7:]), axis=1)
# # inv_yhat = scaler.inverse_transform(inv_yhat)
# inv_yhat = inv_yhat[:,0]
# # invert scaling for actual
# test_y = test_y.reshape((len(test_y), 1))
# inv_y = np.concatenate((test_y, test_X[:, -7:]), axis=1)
# # inv_y = scaler.inverse_transform(inv_y)
# inv_y = inv_y[:,0]
# # calculate RMSE
# rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
# print('Test RMSE: %.3f' % rmse)

In [ ]:
result = pd.DataFrame(inv_yhat,index=x_train.index[:-7])

In [ ]:
a = []
while(1):
    a.append('1')

In [ ]:
result

In [ ]:
ax = plt.plot(inv_y)
ax = plt.plot(inv_yhat)
plt.show()

In [ ]:
match_test_data = retail_test_data.index
hour_converted = []
for i in match_test_data:
  hour_converted.append(i.strftime('%Y-%m-%d %H:00:00'))

new_result = result
new_result['map'] = new_result.index.strftime('%Y-%m-%d %H:00:00')
new_result.columns = ['c','map']
new_result.index = new_result['map']
replace_dict = new_result['c'].to_dict()
retail_test_data['predictions'] = retail_test_data.index.strftime('%Y-%m-%d %H:00:00')
retail_test_data['predictions'].replace(replace_dict,inplace=True)
future_predictions = {'2011-12-09 08:00:00':2.23021,'2011-12-09 09:00:00':2.23021,'2011-12-09 10:00:00':2.23021,'2011-12-09 11:00:00':2.23021,'2011-12-09 12:00:00':2.23021}
retail_test_data['predictions'].replace(future_predictions,inplace=True)


#

In [ ]:
 retail_test_data['predictions'] = np.round(retail_test_data['predictions'].astype(float),decimals=2)
retail_test_data.sort_values('order',inplace=True)
UnitPrice = pd.DataFrame(retail_test_data['predictions'].astype('float').values,columns=['UnitPrice'])
UnitPrice.to_csv('/content/drive/My Drive/TheGreatIndianHiringHackathon/'+'kkcnnlstm05'+'.csv', index=False, index_label=None)

In [ ]:
fg = pd.read_csv('/content/kklstm01.csv')
fg.shape

In [ ]:
c = retail_data.sort_index()
c.tail()

In [ ]:
h = retail_test_data.sort_index()
h.tail(40)

In [ ]:
# #SARIMAX
# model = SARIMAX(y_train, order=(3,1,0),seasonal_order=(1,0,0,7), seasonal=True, enforce_invertibility=False, enforce_stationarity=False,exog=x_train)
# sarimax_results = model.fit()
# predictions = sarimax_results.predict(start=x_train.index[0], end=x_train.index[-1],exog=x_test)
# plt.figure(figsize=(20,5))
# ax = plt.plot(y_train)
# ax = plt.plot(predictions)
# plt.show()
# #One thing that could be done is by distributing the day prediction accross the time stamps of that day. could reduce the error.
# mean_squared_error(y_train,predictions)
# # mean_prediction=predictions
# # mean_prediction.loc[:] = np.round(y_train.mean(),decimals=2)
# # match_predictions(mean_prediction,retail_test_data,'kkmean10')